In [10]:
import pandas as pd

file = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/all-outputs/oneplus-14/claude-sonnet-3.7/android_services_methods_postprocess.parquet"

# read + drop unwanted columns
df = (
    pd.read_parquet(file)
      .drop(columns=[
          "res2", "json_answer2", "output_tokens_1", "input_tokens_1",
          "json_answer", "res1", "sink_code", "to_run",
          "input_tokens", "output_tokens", "depths"
      ])
)

# remove rows that have "invalid" in "access control level predicted"
df = df[~df["access control level predicted"].str.contains("invalid")]
print(df.shape)

# order that reflects increasing strictness
level_rank = {
    "NONE": 0,
    "NORMAL": 1,
    "DANGEROUS": 2,
    "SYS_OR_SIG": 3,   # “signature‑level” permission
}

# keep rows where predicted rank > actual rank
filtered = df[
    df["access control level predicted"].map(level_rank)
    > df["access control level"].map(level_rank)
]

print(filtered["access control level"].value_counts())       # rows/cols that remain
# print(filtered.head())      # quick peek

# save the csv in the same directory as parquet 
filtered.to_csv(file.replace(".parquet", "_mismatches.csv"), index=False)

(462, 5)
access control level
NONE      251
NORMAL     25
Name: count, dtype: int64
